In [1]:
import time
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
from langchain.vectorstores import Chroma
from langchain_community.vectorstores import Marqo
from langchain.vectorstores import LanceDB
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [17]:
import marqo
import json

In [3]:
def readWebsite2(urls):
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return docs


In [4]:
docs = readWebsite2(["https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Philosopher%27s_Stone", "https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Chamber_of_Secrets", "https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Prisoner_of_Azkaban"])

In [5]:
len(docs)

3

In [6]:
def chunkDocs(doc, size):  
    r_text_splitter = RecursiveCharacterTextSplitter(
        # Set custom chunk size
        chunk_size = size,
        chunk_overlap  = 0,
        separators = [ '\n\n', '\n', '\t\t', '\t', ' ', '']
    )
    split = r_text_splitter.split_documents(doc)
    # splits = r_text_splitter.split_text(doc)
    return split 

In [7]:
splits = chunkDocs(docs, 10)

In [8]:
len(splits)

30114

In [9]:
model_name = "sentence-transformers/all-mpnet-base-v2"
# model_name = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

In [10]:
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

C:\Users\linew\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
#takes 10.74 minutes to run with 3 documents and 10 chunks

start = time.time()
persist_directory = 'data/chroma/'
print("create the vector store")
# Create the vector store
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)
print("persist")
vectordb.persist()
end = time.time()
print((end - start)/60)

create the vector store
persist
10.737617794672648


C:\Users\linew\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [37]:
start = time.time()
marqo_url = "http://localhost:8882"  # if using marqo cloud replace with your endpoint (console.marqo.ai)
marqo_api_key = ""  # if using marqo cloud replace with your api key (console.marqo.ai)

client = marqo.Client(url=marqo_url)

index_name = "NoInPutsTestDB"

try:
    client.delete_index(index_name)
except Exception:
    print(f"Creating {index_name}")
    
settings = {"treat_urls_and_pointers_as_images": True, "model": "ViT-L/14"}
client.create_index(index_name)

#docsearch = Marqo.from_documents(splits, index_name=index_name)
client.index(index_name).add_documents(json.dumps(splits))
end = time.time()
print((end - start)/60)

TypeError: Object of type Document is not JSON serializable

In [44]:
start = time.time()
from langchain_community.vectorstores import LanceDB
import lancedb
db = lancedb.connect("/tmp/lancedb")
table = db.create_table(
    "my_table",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)
db = LanceDB.from_documents(splits, embeddings)
end = time.time()
print((end - start)/60)

10.759886566797892
